In [ ]:
# Colab cell (Code)
!pip install -q kaggle==1.5.12
!pip install -q ultralytics==8.0.185  # YOLOv8 (specific version may vary)
!pip install -q torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q pycocotools pandas pillow tqdm matplotlib seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.0/618.0 kB 39.5 MB/s eta 0:00:00


In [ ]:
import os

os.environ["KAGGLE_USERNAME"] = "nikhilsanga1206"
os.environ["KAGGLE_KEY"] = "16c0c673e500366baf8a2262a8494565"

print("Kaggle API Ready!")


Kaggle API Ready!


In [ ]:
# Colab code cell
# Change dataset_slug to the actual Kaggle dataset slug you want to download.
# For your NIH chest x-ray example:
DATASET_SLUG="khanfashee/nih-chest-x-ray-14-224x224-resized"
!mkdir -p /content/datasets
!kaggle datasets download -d $DATASET_SLUG -p /content/datasets --unzip
!ls -lh /content/datasets | sed -n '1,120p'


100% 2.30G/2.30G [01:48<00:00, 24.5MB/s]
100% 2.30G/2.30G [01:48<00:00, 22.8MB/s]
total 38M
-rw-r--r-- 1 root root  91K Nov 14 13:56 BBox_List_2017_Official_NIH.csv
-rw-r--r-- 1 root root 8.3M Nov 14 13:56 Data_Entry_2017.csv
drwxr-xr-x 3 root root 4.0K Nov 14 13:56 images-224
-rw-r--r-- 1 root root  28M Nov 14 13:56 pretrained_model.h5
-rw-r--r-- 1 root root 425K Nov 14 13:56 test_list_NIH.txt
-rw-r--r-- 1 root root 1.5M Nov 14 13:56 train_val_list_NIH.txt


In [ ]:
# Colab code cell
import os, glob
for f in glob.glob('/content/datasets/*'):
    print(f)
# Also list top of dataset folder (images + CSVs)
!ls -lah /content/datasets | sed -n '1,200p'


/content/datasets/Data_Entry_2017.csv
/content/datasets/images-224
/content/datasets/train_val_list_NIH.txt
/content/datasets/BBox_List_2017_Official_NIH.csv
/content/datasets/test_list_NIH.txt
/content/datasets/pretrained_model.h5
total 38M
drwxr-xr-x 3 root root 4.0K Nov 14 13:56 .
drwxr-xr-x 1 root root 4.0K Nov 14 13:54 ..
-rw-r--r-- 1 root root  91K Nov 14 13:56 BBox_List_2017_Official_NIH.csv
-rw-r--r-- 1 root root 8.3M Nov 14 13:56 Data_Entry_2017.csv
drwxr-xr-x 3 root root 4.0K Nov 14 13:56 images-224
-rw-r--r-- 1 root root  28M Nov 14 13:56 pretrained_model.h5
-rw-r--r-- 1 root root 425K Nov 14 13:56 test_list_NIH.txt
-rw-r--r-- 1 root root 1.5M Nov 14 13:56 train_val_list_NIH.txt


In [ ]:
# Colab code cell
import pandas as pd
META = "/content/datasets/Data_Entry_2017.csv"
BBOX = "/content/datasets/BBox_List_2017_Official_NIH.csv"
print("Files exist?", pd.io.common.file_exists(META), pd.io.common.file_exists(BBOX))

meta_df = pd.read_csv(META)
bbox_df = pd.read_csv(BBOX)

# Clean column names
meta_df.columns = meta_df.columns.str.strip().str.lower().str.replace(" ", "_")
bbox_df.columns = bbox_df.columns.str.strip().str.lower().str.replace(" ", "_")

print("meta cols:", meta_df.columns.tolist()[:10])
print("bbox cols:", bbox_df.columns.tolist()[:20])
print("Sample bbox rows:")
bbox_df.head()


Files exist? True True
meta cols: ['image_index', 'finding_labels', 'follow-up_#', 'patient_id', 'patient_age', 'patient_gender', 'view_position', 'originalimage[width', 'height]', 'originalimagepixelspacing[x']
bbox cols: ['image_index', 'finding_label', 'bbox_[x', 'y', 'w', 'h]', 'unnamed:_6', 'unnamed:_7', 'unnamed:_8']
Sample bbox rows:


,image_index,finding_label,bbox_[x,y,w,h],unnamed:_6,unnamed:_7,unnamed:_8
0,00013118_008.png,Atelectasis,225.084746,547.019217,86.779661,79.186441,NaN,NaN,NaN
1,00014716_007.png,Atelectasis,686.101695,131.543498,185.491525,313.491525,NaN,NaN,NaN
2,00029817_009.png,Atelectasis,221.830508,317.053115,155.118644,216.949153,NaN,NaN,NaN
3,00014687_001.png,Atelectasis,726.237288,494.951420,141.016949,55.322034,NaN,NaN,NaN
4,00017877_001.png,Atelectasis,660.067797,569.780787,200.677966,78.101695,NaN,NaN,NaN


In [ ]:
import json, os
from pathlib import Path

IMG_DIR = Path("/content/datasets/images-224/images-224")
COCO_OUT = "/content/coco_chestxray.json"

bbox_df_clean = bbox_df.copy()

# COLUMN NAMES (after cleaning)
img_col = "image_index"
label_col = "finding_label"

# Correct NIH Bounding Box Columns
x1_col = "bbox_x1"
y1_col = "bbox_y1"
x2_col = "bbox_x2"
y2_col = "bbox_y2"

# Build categories
unique_labels = sorted(bbox_df_clean[label_col].unique().tolist())
categories = [{"id": i+1, "name": name} for i, name in enumerate(unique_labels)]
label2id = {name: i+1 for i, name in enumerate(unique_labels)}

images = []
annotations = []
img_id_map = {}
next_img_id = 1
next_ann_id = 1

for _, row in bbox_df_clean.iterrows():
    fname = str(row[img_col])
    label = row[label_col]

    # Extract bounding box
    try:
        x1 = float(row[x1_col])
        y1 = float(row[y1_col])
        x2 = float(row[x2_col])
        y2 = float(row[y2_col])
    except:
        continue

    w = x2 - x1
    h = y2 - y1

    # Add image entry if not already
    if fname not in img_id_map:
        img_id_map[fname] = next_img_id
        images.append({
            "id": next_img_id,
            "file_name": fname,
            "width": 224,
            "height": 224
        })
        next_img_id += 1

    # Add annotation
    annotations.append({
        "id": next_ann_id,
        "image_id": img_id_map[fname],
        "category_id": label2id[label],
        "bbox": [x1, y1, w, h],
        "area": float(w * h),
        "iscrowd": 0
    })
    next_ann_id += 1

# Final COCO dictionary
coco = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

# Save JSON
with open(COCO_OUT, "w") as f:
    json.dump(coco, f, indent=4)

print("Wrote COCO:", COCO_OUT)
print("Images:", len(images))
print("Annotations:", len(annotations))
print("Categories:", len(categories))


Wrote COCO: /content/coco_chestxray.json
Images: 0
Annotations: 0
Categories: 8


In [ ]:
bbox_df = pd.read_csv("/content/datasets/BBox_List_2017_Official_NIH.csv")
bbox_df.columns = bbox_df.columns.str.strip().str.lower().str.replace(" ", "_")
bbox_df.columns.tolist()


['image_index',
 'finding_label',
 'bbox_[x',
 'y',
 'w',
 'h]',
 'unnamed:_6',
 'unnamed:_7',
 'unnamed:_8']

In [ ]:
import json, os
from pathlib import Path

IMG_DIR = Path("/content/datasets/images-224/images-224")
COCO_OUT = "/content/coco_chestxray.json"

bbox_df_clean = bbox_df.copy()

# Correct columns (confirmed)
img_col = "image_index"
label_col = "finding_label"
x_col = "bbox_[x"
y_col = "y"
w_col = "w"
h_col = "h]"

# Build categories dictionary
unique_labels = sorted(bbox_df_clean[label_col].unique().tolist())
categories = [{"id": i+1, "name": name} for i, name in enumerate(unique_labels)]
label2id = {name: i+1 for i, name in enumerate(unique_labels)}

images = []
annotations = []
img_id_map = {}
next_img_id = 1
next_ann_id = 1

for _, row in bbox_df_clean.iterrows():

    fname = str(row[img_col])
    label = row[label_col]

    # Get bbox values
    try:
        x = float(row[x_col])
        y = float(row[y_col])
        w = float(row[w_col])
        h = float(row[h_col])
    except:
        continue

    # Add image entry once
    if fname not in img_id_map:
        img_id_map[fname] = next_img_id
        images.append({
            "id": next_img_id,
            "file_name": fname,
            "width": 224,
            "height": 224
        })
        next_img_id += 1

    # Add annotation
    annotations.append({
        "id": next_ann_id,
        "image_id": img_id_map[fname],
        "category_id": label2id[label],
        "bbox": [x, y, w, h],
        "area": w * h,
        "iscrowd": 0
    })
    next_ann_id += 1

# Save COCO dict
coco = {
    "images": images,
    "annotations": annotations,
    "categories": categories
}

with open(COCO_OUT, "w") as f:
    json.dump(coco, f, indent=4)

print("Wrote COCO:", COCO_OUT)
print("Images:", len(images))
print("Annotations:", len(annotations))
print("Categories:", len(categories))


Wrote COCO: /content/coco_chestxray.json
Images: 880
Annotations: 984
Categories: 8


In [ ]:
import json, random, shutil
from pathlib import Path

# Paths
COCO_JSON = "/content/coco_chestxray.json"
SRC_IMG_DIR = Path("/content/datasets/images-224/images-224")

OUT_ROOT = Path("/content/coco_dataset")
IMG_TRAIN = OUT_ROOT / "images/train"
IMG_VAL = OUT_ROOT / "images/val"
ANN_DIR = OUT_ROOT / "annotations"

# Create folders
IMG_TRAIN.mkdir(parents=True, exist_ok=True)
IMG_VAL.mkdir(parents=True, exist_ok=True)
ANN_DIR.mkdir(parents=True, exist_ok=True)

# Load COCO JSON
with open(COCO_JSON, "r") as f:
    coco = json.load(f)

images = coco["images"]
annotations = coco["annotations"]
categories = coco["categories"]

# Shuffle & split
random.seed(42)
image_ids = [img["id"] for img in images]
random.shuffle(image_ids)

split = int(0.90 * len(image_ids))  # 90% train
train_ids = set(image_ids[:split])
val_ids = set(image_ids[split:])

# Separate images and annotations
train_images = [img for img in images if img["id"] in train_ids]
val_images   = [img for img in images if img["id"] in val_ids]

train_ann = [ann for ann in annotations if ann["image_id"] in train_ids]
val_ann   = [ann for ann in annotations if ann["image_id"] in val_ids]

# Copy images
missing = 0
for img in images:
    fname = img["file_name"]
    src_path = SRC_IMG_DIR / fname

    if not src_path.exists():
        missing += 1
        continue

    if img["id"] in train_ids:
        shutil.copy(src_path, IMG_TRAIN / fname)
    else:
        shutil.copy(src_path, IMG_VAL / fname)

# Save annotation files
with open(ANN_DIR / "instances_train.json", "w") as f:
    json.dump({"images": train_images, "annotations": train_ann, "categories": categories}, f, indent=4)

with open(ANN_DIR / "instances_val.json", "w") as f:
    json.dump({"images": val_images, "annotations": val_ann, "categories": categories}, f, indent=4)

print("Train images:", len(train_images))
print("Val images:", len(val_images))
print("Train annotations:", len(train_ann))
print("Val annotations:", len(val_ann))
print("Missing images:", missing)
print("Dataset created at:", OUT_ROOT)


Train images: 792
Val images: 88
Train annotations: 888
Val annotations: 96
Missing images: 0
Dataset created at: /content/coco_dataset


In [ ]:
from pathlib import Path
import json

root = Path("/content/coco_dataset")
data_yaml = root / "data.yaml"

# category names
with open("/content/coco_chestxray.json", "r") as f:
    coco_json = json.load(f)

names = [cat["name"] for cat in coco_json["categories"]]
nc = len(names)

data_yaml.write_text(
    f"path: {root}\n"
    f"train: images/train\n"
    f"val: images/val\n"
    f"nc: {nc}\n"
    f"names: {names}\n"
)

print("Created:", data_yaml)
print(data_yaml.read_text())


Created: /content/coco_dataset/data.yaml
path: /content/coco_dataset
train: images/train
val: images/val
nc: 8
names: ['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltrate', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']



In [ ]:
import torch.serialization
import ultralytics.nn.tasks as tasks
import inspect

# Find ALL classes inside ultralytics.nn.tasks
safe_list = []
for name, obj in tasks.__dict__.items():
    if inspect.isclass(obj):
        safe_list.append(obj)

torch.serialization.add_safe_globals(safe_list)

print("Whitelisted YOLO model classes:", [cls.__name__ for cls in safe_list])
print("Safe globals added successfully.")


Whitelisted YOLO model classes: ['Path', 'AIFI', 'C1', 'C2', 'C3', 'C3TR', 'SPP', 'SPPF', 'Bottleneck', 'BottleneckCSP', 'C2f', 'C3Ghost', 'C3x', 'Classify', 'Concat', 'Conv', 'Conv2', 'ConvTranspose', 'Detect', 'DWConv', 'DWConvTranspose2d', 'Focus', 'GhostBottleneck', 'GhostConv', 'HGBlock', 'HGStem', 'Pose', 'RepC3', 'RepConv', 'RTDETRDecoder', 'Segment', 'v8ClassificationLoss', 'v8DetectionLoss', 'v8PoseLoss', 'v8SegmentationLoss', 'BaseModel', 'DetectionModel', 'SegmentationModel', 'PoseModel', 'ClassificationModel', 'RTDETRDetectionModel', 'Ensemble']
Safe globals added successfully.


In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 --index-url https://download.pytorch.org/whl/cu118
!pip install ultralytics==8.1.34


Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.7/857.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 108.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 118.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/66

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 723.1/723.1 kB 43.6 MB/s eta 0:00:00
ERROR: Operation cancelled by user
^C


In [ ]:
from ultralytics import YOLO
print("YOLO imported successfully!")


YOLO imported successfully!


In [ ]:
!yolo detect train model=yolov8n.pt data=/content/coco_dataset/data.yaml epochs=10 imgsz=640 batch=16 project=/content/yolo_train name=exp


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
print("W&B disabled successfully.")


W&B disabled successfully.


In [ ]:
!yolo detect train model=yolov8n.pt data=/content/coco_dataset/data.yaml epochs=10 imgsz=640 batch=16 project=yolo_train name=exp wandb=False


Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/__init__.py", line 385, in entrypoint
    check_dict_alignment(full_args_dict, overrides)
  File "/usr/local/lib/python3.12/dist-packages/ultralytics/cfg/__init__.py", line 203, in check_dict_alignment
    raise SyntaxError(string + CLI_HELP_MSG) from e
SyntaxError: 'wandb' is not a valid YOLO argument. 

    Arguments received: ['yolo', 'detect', 'train', 'model=yolov8n.pt', 'data=/content/coco_dataset/data.yaml', 'epochs=10', 'imgsz=640', 'batch=16', 'project=yolo_train', 'name=exp', 'wandb=False']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
                ARGS 

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
print("WANDB DISABLED")


WANDB DISABLED


In [ ]:
!yolo settings wandb=False


💡 Learn about settings at https://docs.ultralytics.com/quickstart/#ultralytics-settings
Printing '/root/.config/Ultralytics/settings.yaml'

settings_version: 0.0.4
datasets_dir: /content/datasets
weights_dir: weights
runs_dir: runs
uuid: 569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb
sync: true
api_key: ''
clearml: true
comet: true
dvc: true
hub: true
mlflow: true
neptune: true
raytune: true
tensorboard: true
wandb: false



In [ ]:
import json
import os
from pathlib import Path

COCO_ROOT = Path("/content/coco_dataset")
TRAIN_JSON = COCO_ROOT / "annotations/instances_train.json"
VAL_JSON = COCO_ROOT / "annotations/instances_val.json"

LABEL_TRAIN = COCO_ROOT / "labels/train"
LABEL_VAL = COCO_ROOT / "labels/val"

LABEL_TRAIN.mkdir(parents=True, exist_ok=True)
LABEL_VAL.mkdir(parents=True, exist_ok=True)

def coco_to_yolo(coco_json_path, label_output_path):
    with open(coco_json_path, "r") as f:
        coco = json.load(f)

    images = {img["id"]: img["file_name"] for img in coco["images"]}
    image_sizes = {img["id"]: (img["width"], img["height"]) for img in coco["images"]}

    anns = coco["annotations"]

    # clear directory
    for txt in label_output_path.glob("*.txt"):
        txt.unlink()

    for ann in anns:
        img_id = ann["image_id"]
        file_name = images[img_id]
        img_w, img_h = image_sizes[img_id]

        # YOLO bbox format
        x, y, w, h = ann["bbox"]
        xc = x + w / 2
        yc = y + h / 2

        # normalize
        xc /= img_w
        yc /= img_h
        w /= img_w
        h /= img_h

        cls = ann["category_id"] - 1  # YOLO class ids start at 0

        txt_path = label_output_path / (file_name.replace('.png','').replace('.jpg','') + ".txt")
        with open(txt_path, "a") as f:
            f.write(f"{cls} {xc} {yc} {w} {h}\n")

    print(f"YOLO labels written to: {label_output_path}")

# Convert both splits
coco_to_yolo(TRAIN_JSON, LABEL_TRAIN)
coco_to_yolo(VAL_JSON, LABEL_VAL)

print("Conversion complete.")


YOLO labels written to: /content/coco_dataset/labels/train
YOLO labels written to: /content/coco_dataset/labels/val
Conversion complete.


In [ ]:
!ls /content/coco_dataset/labels/train | head
!ls /content/coco_dataset/labels/train | wc -l
!ls /content/coco_dataset/labels/val | wc -l


00000032_037.txt
00000072_000.txt
00000147_001.txt
00000149_006.txt
00000150_002.txt
00000181_061.txt
00000193_019.txt
00000211_010.txt
00000211_016.txt
00000211_019.txt
792
88


In [ ]:
from PIL import Image
import os
from pathlib import Path

root = Path("/content/datasets/images-224/images-224")
sizes = set()

for i, fname in enumerate(os.listdir(root)):
    if i > 200:
        break
    img = Image.open(root / fname)
    sizes.add(img.size)

sizes


{(224, 224)}

In [ ]:
import json
from pathlib import Path
from PIL import Image

COCO_ROOT = Path("/content/coco_dataset")
TRAIN_JSON = COCO_ROOT / "annotations/instances_train.json"
VAL_JSON   = COCO_ROOT / "annotations/instances_val.json"

LABEL_TRAIN = COCO_ROOT / "labels/train"
LABEL_VAL   = COCO_ROOT / "labels/val"

LABEL_TRAIN.mkdir(parents=True, exist_ok=True)
LABEL_VAL.mkdir(parents=True, exist_ok=True)

def convert_coco_to_yolo(json_file, label_dir, image_root):
    with open(json_file, 'r') as f:
        coco = json.load(f)

    images = {img["id"]: img for img in coco["images"]}

    # Clear old TXT files
    for txt in label_dir.glob("*.txt"):
        txt.unlink()

    for ann in coco["annotations"]:
        img_info = images[ann["image_id"]]
        img_file = img_info["file_name"]

        # Open image to read REAL width/height
        img_path = image_root / img_file
        try:
            with Image.open(img_path) as im:
                w_img, h_img = im.size
        except:
            continue

        x, y, w, h = ann["bbox"]

        # YOLO format
        xc = (x + w/2) / w_img
        yc = (y + h/2) / h_img
        wn = w / w_img
        hn = h / h_img

        # Skip invalid (YOLO requires all in 0-1 range)
        if not (0 <= xc <= 1 and 0 <= yc <= 1 and 0 <= wn <= 1 and 0 <= hn <= 1):
            continue

        cls = ann["category_id"] - 1

        label_path = label_dir / f"{img_file.rsplit('.',1)[0]}.txt"
        with open(label_path, "a") as f:
            f.write(f"{cls} {xc} {yc} {wn} {hn}\n")

    print(f"Converted: {json_file} → {label_dir}")

IMG_DIR = Path("/content/coco_dataset/images/train")
convert_coco_to_yolo(TRAIN_JSON, LABEL_TRAIN, IMG_DIR)

IMG_DIR = Path("/content/coco_dataset/images/val")
convert_coco_to_yolo(VAL_JSON, LABEL_VAL, IMG_DIR)


Converted: /content/coco_dataset/annotations/instances_train.json → /content/coco_dataset/labels/train
Converted: /content/coco_dataset/annotations/instances_val.json → /content/coco_dataset/labels/val


In [ ]:
!ls /content/coco_dataset/labels/train | wc -l
!ls /content/coco_dataset/labels/val | wc -l


0
0


In [ ]:
# Cell A - backup existing COCO and labels, set scale
from pathlib import Path
import shutil, json, os

ROOT = Path("/content")
COCO_ORIG = ROOT / "coco_chestxray.json"
COCO_SCALED = ROOT / "coco_chestxray_scaled.json"
COCO_DATASET = ROOT / "coco_dataset"

# Backup old files
if COCO_ORIG.exists():
    shutil.copy(str(COCO_ORIG), str(COCO_ORIG.with_suffix(".bak.json")))
    print("Backed up original COCO to", COCO_ORIG.with_suffix(".bak.json"))

# Parameters
ORIG_SIZE = 1024.0   # you told: take 1024 (original image size)
NEW_SIZE = 224.0     # your current images are 224
SCALE_X = NEW_SIZE / ORIG_SIZE
SCALE_Y = NEW_SIZE / ORIG_SIZE
print("Scale factors:", SCALE_X, SCALE_Y)


Backed up original COCO to /content/coco_chestxray.bak.json
Scale factors: 0.21875 0.21875


In [ ]:
# Cell B - scale bboxes and write new COCO JSON that aligns to 224x224 images
import json

COCO_IN = "/content/coco_chestxray.json"
COCO_OUT = "/content/coco_chestxray_scaled.json"

ORIG_SIZE = 1024.0
NEW_SIZE = 224.0
SCALE_X = NEW_SIZE / ORIG_SIZE
SCALE_Y = NEW_SIZE / ORIG_SIZE

with open(COCO_IN, "r") as f:
    coco = json.load(f)

images = coco.get("images", [])
anns = coco.get("annotations", [])
cats = coco.get("categories", [])

new_images = []
new_anns = []
next_ann_id = 1

# update image sizes to 224x224
for img in images:
    new_images.append({
        "id": img['id'],
        "file_name": img['file_name'],
        "width": int(NEW_SIZE),
        "height": int(NEW_SIZE)
    })

# scale and clip bounding boxes
for ann in anns:
    x, y, w, h = ann['bbox']

    xs = x * SCALE_X
    ys = y * SCALE_Y
    ws = w * SCALE_X
    hs = h * SCALE_Y

    # clip to bounds
    xs = max(0, min(xs, NEW_SIZE-1))
    ys = max(0, min(ys, NEW_SIZE-1))
    ws = max(1, min(ws, NEW_SIZE-xs))
    hs = max(1, min(hs, NEW_SIZE-ys))

    if ws < 1 or hs < 1:
        continue

    new_anns.append({
        "id": next_ann_id,
        "image_id": ann["image_id"],
        "category_id": ann["category_id"],
        "bbox": [xs, ys, ws, hs],
        "area": float(ws * hs),
        "iscrowd": ann.get("iscrowd", 0)
    })
    next_ann_id += 1

scaled_coco = {
    "images": new_images,
    "annotations": new_anns,
    "categories": cats
}

with open(COCO_OUT, "w") as f:
    json.dump(scaled_coco, f, indent=2)

print("Saved SCALED COCO to:", COCO_OUT)
print("Images:", len(new_images))
print("Annotations:", len(new_anns))
print("Categories:", len(cats))


Saved SCALED COCO to: /content/coco_chestxray_scaled.json
Images: 880
Annotations: 984
Categories: 8


In [ ]:
# Cell C - create coco_dataset (images already present) using SCALED COCO
import json, random, shutil
from pathlib import Path

COCO_SCALED = "/content/coco_chestxray_scaled.json"
SRC_IMG_DIR = Path("/content/datasets/images-224/images-224")

OUT_ROOT = Path("/content/coco_dataset")
IMG_TRAIN = OUT_ROOT / "images/train"
IMG_VAL = OUT_ROOT / "images/val"
ANN_DIR = OUT_ROOT / "annotations"

# make folders
IMG_TRAIN.mkdir(parents=True, exist_ok=True)
IMG_VAL.mkdir(parents=True, exist_ok=True)
ANN_DIR.mkdir(parents=True, exist_ok=True)

# load scaled COCO
with open(COCO_SCALED, "r") as f:
    coco = json.load(f)

images = coco["images"]
annotations = coco["annotations"]
categories = coco["categories"]

random.seed(42)
image_ids = [img["id"] for img in images]
random.shuffle(image_ids)

split = int(0.90 * len(image_ids))   # 90% train
train_ids = set(image_ids[:split])
val_ids = set(image_ids[split:])

train_images = [img for img in images if img["id"] in train_ids]
val_images   = [img for img in images if img["id"] in val_ids]

train_ann = [ann for ann in annotations if ann["image_id"] in train_ids]
val_ann   = [ann for ann in annotations if ann["image_id"] in val_ids]

missing = 0

# copy images
for img in images:
    fname = img["file_name"]
    src = SRC_IMG_DIR / fname
    if not src.exists():
        missing += 1
        continue
    dst = IMG_TRAIN if img["id"] in train_ids else IMG_VAL
    shutil.copy(str(src), str(dst / fname))

# write annotation files
with open(ANN_DIR / "instances_train.json", "w") as f:
    json.dump({"images": train_images, "annotations": train_ann, "categories": categories}, f, indent=2)

with open(ANN_DIR / "instances_val.json", "w") as f:
    json.dump({"images": val_images, "annotations": val_ann, "categories": categories}, f, indent=2)

print("Train images:", len(train_images))
print("Val images:", len(val_images))
print("Train annotations:", len(train_ann))
print("Val annotations:", len(val_ann))
print("Missing images:", missing)
print("Scaled dataset created at:", OUT_ROOT)


Train images: 792
Val images: 88
Train annotations: 888
Val annotations: 96
Missing images: 0
Scaled dataset created at: /content/coco_dataset


In [ ]:
# Cell D - Convert scaled COCO annotations to YOLO TXT labels
import json
from pathlib import Path
from PIL import Image

COCO_ROOT = Path("/content/coco_dataset")
TRAIN_JSON = COCO_ROOT / "annotations/instances_train.json"
VAL_JSON   = COCO_ROOT / "annotations/instances_val.json"

LABEL_TRAIN = COCO_ROOT / "labels/train"
LABEL_VAL   = COCO_ROOT / "labels/val"

LABEL_TRAIN.mkdir(parents=True, exist_ok=True)
LABEL_VAL.mkdir(parents=True, exist_ok=True)

def convert_coco_to_yolo(json_file, label_output_path, image_root):
    with open(json_file, "r") as f:
        coco = json.load(f)

    images = {img["id"]: img for img in coco["images"]}

    # clear old txt files
    for t in label_output_path.glob("*.txt"):
        t.unlink()

    total_written = 0

    for ann in coco["annotations"]:
        img_info = images[ann["image_id"]]
        img_file = img_info["file_name"]

        img_path = image_root / img_file

        try:
            with Image.open(img_path) as im:
                w_img, h_img = im.size
        except:
            continue

        x, y, w, h = ann["bbox"]

        # convert to YOLO format
        xc = (x + w/2) / w_img
        yc = (y + h/2) / h_img
        wn = w / w_img
        hn = h / h_img

        # skip invalid normalized coords
        if not (0 <= xc <= 1 and 0 <= yc <= 1 and 0 <= wn <= 1 and 0 <= hn <= 1):
            continue

        cls = ann["category_id"] - 1

        # write txt
        txt_path = label_output_path / f"{img_file.rsplit('.', 1)[0]}.txt"
        with open(txt_path, "a") as f:
            f.write(f"{cls} {xc:.6f} {yc:.6f} {wn:.6f} {hn:.6f}\n")
        total_written += 1

    print(f"Converted {total_written} annotations to YOLO format in {label_output_path}")

# convert train
convert_coco_to_yolo(
    TRAIN_JSON,
    LABEL_TRAIN,
    COCO_ROOT / "images/train"
)

# convert val
convert_coco_to_yolo(
    VAL_JSON,
    LABEL_VAL,
    COCO_ROOT / "images/val"
)


Converted 888 annotations to YOLO format in /content/coco_dataset/labels/train
Converted 96 annotations to YOLO format in /content/coco_dataset/labels/val


In [ ]:
!ls /content/coco_dataset/labels/train | wc -l
!ls /content/coco_dataset/labels/val | wc -l


792
88


In [ ]:
!yolo detect train model=yolov8n.pt data=/content/coco_dataset/data.yaml epochs=10 imgsz=640 batch=16 project=/content/yolo_train name=exp_final


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


mAP50      = 0.209
mAP50-95   = 0.122
Precision  = 0.776
Recall     = 0.188


In [ ]:
%%writefile train_faster_rcnn.py
import torch
import torchvision
import torchvision.transforms as T
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import json
import os
from pycocotools.coco import COCO
import numpy as np
from tqdm import tqdm

# ---------------------------
# CUSTOM COCO DATASET
# ---------------------------
class CocoDetection(Dataset):
    def __init__(self, img_folder, ann_file, transforms=None):
        self.coco = COCO(ann_file)
        self.img_folder = img_folder
        self.transforms = transforms
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        img_id = self.ids[index]
        ann_ids = self.coco.getAnnIds(imgIds=img_id)

        anns = self.coco.loadAnns(ann_ids)
        img_info = self.coco.loadImgs(img_id)[0]
        path = img_info["file_name"]

        # load image
        img_path = os.path.join(self.img_folder, path)
        img = Image.open(img_path).convert("RGB")

        # extract bounding boxes
        boxes = []
        labels = []
        areas = []
        iscrowd = []

        for ann in anns:
            x, y, w, h = ann["bbox"]
            if w <= 0 or h <= 0:
                continue

            boxes.append([x, y, x + w, y + h])
            labels.append(ann["category_id"])
            areas.append(ann["area"])
            iscrowd.append(ann.get("iscrowd", 0))

        if len(boxes) == 0:
            # If no bbox exists, give dummy box (FRCNN NEEDS A BOX!)
            boxes = [[0, 0, 1, 1]]
            labels = [1]
            areas = [1.0]
            iscrowd = [0]

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        areas = torch.as_tensor(areas, dtype=torch.float32)
        iscrowd = torch.as_tensor(iscrowd)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["area"] = areas
        target["iscrowd"] = iscrowd
        target["image_id"] = torch.tensor([img_id])

        if self.transforms:
            img = self.transforms(img)

        return img, target

    def __len__(self):
        return len(self.ids)


# ---------------------------
# COLLATE FN
# ---------------------------
def collate_fn(batch):
    return tuple(zip(*batch))


# ---------------------------
# EVALUATION FUNCTION
# ---------------------------
def evaluate(model, dataloader, ann_file, device):
    print("Evaluating...")
    coco_gt = COCO(ann_file)

    model.eval()
    results = []

    with torch.no_grad():
        for imgs, targets in tqdm(dataloader):
            imgs = [img.to(device) for img in imgs]
            outputs = model(imgs)

            for target, output in zip(targets, outputs):
                image_id = int(target["image_id"].item())

                if "boxes" not in output:
                    continue

                boxes = output["boxes"].cpu().numpy()
                scores = output["scores"].cpu().numpy()
                labels = output["labels"].cpu().numpy()

                for box, score, label in zip(boxes, scores, labels):
                    x1, y1, x2, y2 = box
                    w = x2 - x1
                    h = y2 - y1
                    results.append({
                        "image_id": image_id,
                        "category_id": int(label),
                        "bbox": [float(x1), float(y1), float(w), float(h)],
                        "score": float(score)
                    })

    # Save temp JSON
    import tempfile
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".json")
    import json
    json.dump(results, open(tmp.name, "w"))

    coco_dt = coco_gt.loadRes(tmp.name)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    return float(coco_eval.stats[1])  # mAP50


# ---------------------------
# TRAINING LOOP
# ---------------------------
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Device:", device)

    # Paths
    train_imgs = "/content/coco_dataset/images/train"
    val_imgs = "/content/coco_dataset/images/val"
    train_ann = "/content/coco_dataset/annotations/instances_train.json"
    val_ann = "/content/coco_dataset/annotations/instances_val.json"

    # Transforms
    transform = T.Compose([T.ToTensor()])

    # Dataset + Loader
    train_dataset = CocoDetection(train_imgs, train_ann, transforms=transform)
    val_dataset   = CocoDetection(val_imgs, val_ann, transforms=transform)

    train_dl = DataLoader(train_dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)
    val_dl   = DataLoader(val_dataset, batch_size=4, shuffle=False, collate_fn=collate_fn)

    # Model
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    model.to(device)

    # Optimizer
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(params, lr=0.0001)

    best_map50 = 0.0

    for epoch in range(1, 6):  # train 5 epochs
        print(f"\n--- Epoch {epoch} ---")

        model.train()
        total_loss = 0.0

        for imgs, targets in tqdm(train_dl):
            imgs = [img.to(device) for img in imgs]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(imgs, targets)
            losses = sum(loss_dict.values())

            optimizer.zero_grad()
            losses.backward()
            optimizer.step()

            total_loss += losses.item()

        print(f"Epoch {epoch} Loss: {total_loss:.4f}")

        # Validate
        map50 = evaluate(model, val_dl, val_ann, device)
        print("mAP50:", map50)

        if map50 > best_map50:
            best_map50 = map50
            torch.save(model.state_dict(), "/content/faster_rcnn_best.pth")
            print("Saved NEW BEST model!")

    print("Training Completed.")
    print("Best mAP50:", best_map50)


if __name__ == "__main__":
    train()


Overwriting train_faster_rcnn.py


In [ ]:
!python train_faster_rcnn.py


Device: cuda
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

--- Epoch 1 ---
100% 198/198 [02:28<00:00,  1.33it/s]
Epoch 1 Loss: 52.1369
Evaluating...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
100% 22/22 [00:07<00:00,  2.98it/s]
Traceback (most recent call last):
  File "/content/train_faster_rcnn.py", line 203, in <module>
    train()
  File "/content/train_faster_rcnn.py", line 190, in train
    map50 = evaluate(model, val_dl, val_ann, device)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/content/train_faster_rcnn.py", line 127, in evaluate
    coco_dt = coco_gt.loadRes(tmp.name)
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pycocotools/coco.py", line 314, in loadRes
    res.dataset['info'] = copy.deepcopy(self.dataset['info'])
                                        ~~

In [ ]:
from ultralytics import YOLO

model = YOLO("/content/yolo_train/exp_final/weights/best.pt")
metrics = model.val(data="/content/coco_dataset/data.yaml")


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
!yolo val model=/content/yolo_train/exp_final/weights/best.pt data=/content/coco_dataset/data.yaml imgsz=640


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
!yolo detect predict model=/content/yolo_train/exp_final/weights/best.pt source=/content/input.png conf=0.25 save=True


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
!yolo detect predict model=/content/yolo_train/exp_final/weights/best.pt source=/content/coco_dataset/images/val save=True


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
!yolo detect train model=yolov8n.pt data=/content/coco_dataset/data.yaml epochs=50 imgsz=640 batch=16 project=yolo_train name=exp_long


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
!yolo export model=/content/yolo_train/exp_final/weights/best.pt format=onnx


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
!yolo export model=/content/yolo_train/exp_final/weights/best.pt format=engine


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


In [ ]:
from ultralytics import YOLO
model = YOLO("best.pt")
results = model("xray.png")
results.show()


/usr/local/lib/python3.12/dist-packages/ultralytics/nn/tasks.py:527: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


FileNotFoundError: [Errno 2] No such file or directory: 'best.pt'

In [ ]:
from google.colab import files
files.download("/content/yolo_train/exp_final/weights/best.pt")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import json

def fix_coco(json_path):
    with open(json_path, "r") as f:
        coco = json.load(f)

    # Add required COCO fields if missing
    if "info" not in coco:
        coco["info"] = {
            "description": "NIH Chest X-ray Detection Dataset",
            "version": "1.0",
            "year": 2025
        }

    if "licenses" not in coco:
        coco["licenses"] = [{
            "id": 1,
            "name": "NIH License",
            "url": "https://nih.gov"
        }]

    # save back
    with open(json_path, "w") as f:
        json.dump(coco, f, indent=2)

    print("Fixed COCO:", json_path)

fix_coco("/content/coco_dataset/annotations/instances_train.json")
fix_coco("/content/coco_dataset/annotations/instances_val.json")


Fixed COCO: /content/coco_dataset/annotations/instances_train.json
Fixed COCO: /content/coco_dataset/annotations/instances_val.json


In [ ]:
%%writefile train_faster_rcnn.py
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
import torchvision.transforms as T
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import tempfile
import json
import numpy as np

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(batch):
    return tuple(zip(*batch))

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

def evaluate(model, dataloader, ann_file, device):
    model.eval()
    coco_gt = COCO(ann_file)
    results = []

    with torch.no_grad():
        for imgs, targets in dataloader:
            imgs = [img.to(device) for img in imgs]
            outputs = model(imgs)

            for target, output in zip(targets, outputs):
                image_id = target["image_id"].item()

                for box, score, label in zip(output["boxes"], output["scores"], output["labels"]):
                    x1, y1, x2, y2 = box.tolist()
                    results.append({
                        "image_id": image_id,
                        "category_id": int(label),
                        "bbox": [x1, y1, x2 - x1, y2 - y1],
                        "score": float(score)
                    })

    # Save predictions
    with tempfile.NamedTemporaryFile(mode="w", delete=False) as f:
        json.dump(results, f)
        temp_path = f.name

    coco_dt = coco_gt.loadRes(temp_path)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval.stats[1]  # mAP50

def train():
    train_ann = "/content/coco_dataset/annotations/instances_train.json"
    val_ann = "/content/coco_dataset/annotations/instances_val.json"
    train_img = "/content/coco_dataset/images/train"
    val_img = "/content/coco_dataset/images/val"

    trans = T.Compose([T.ToTensor()])

    train_ds = CocoDetection(train_img, train_ann, transforms=trans)
    val_ds   = CocoDetection(val_img, val_ann, transforms=trans)

    train_dl = DataLoader(train_ds, batch_size=4, shuffle=True,
                          num_workers=2, collate_fn=collate_fn)
    val_dl   = DataLoader(val_ds, batch_size=4, shuffle=False,
                          num_workers=2, collate_fn=collate_fn)

    num_classes = 1 + 8  # background + 8 classes
    model = get_model(num_classes).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    best_map50 = 0

    for epoch in range(5):
        print(f"Epoch {epoch+1}")

        model.train()
        for imgs, targets in train_dl:
            imgs = [img.to(DEVICE) for img in imgs]
            target_list = []
            for i, t in enumerate(targets):
                boxes = torch.tensor(t["boxes"], dtype=torch.float32).to(DEVICE)
                labels = torch.tensor(t["labels"], dtype=torch.int64).to(DEVICE)

                target_list.append({"boxes": boxes, "labels": labels})

            loss_dict = model(imgs, target_list)
            loss = sum(loss_dict.values())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Evaluating...")
        map50 = evaluate(model, val_dl, val_ann, DEVICE)

        if map50 > best_map50:
            best_map50 = map50
            torch.save(model.state_dict(), "/content/faster_rcnn_best.pth")
            print("Saved BEST model!")

    print("Training finished.")
    print("BEST mAP50:", best_map50)

if __name__ == "__main__":
    train()


Overwriting train_faster_rcnn.py


In [ ]:
!python train_faster_rcnn.py


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Epoch 1
Traceback (most recent call last):
  File "/content/train_faster_rcnn.py", line 116, in <module>
    train()
  File "/content/train_faster_rcnn.py", line 88, in train
    for imgs, targets in train_dl:
                         ^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 630, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1344, in _next_data
    return self._process_data(data)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1370, in _process_data
    data.reraise()
  File "/usr/local/lib/python3.12/dist-packages/torch/_utils.py", line 706, in reraise
    raise exception
TypeError: Caugh

In [ ]:
def get_transform():
    return T.Compose([
        T.ToTensor()
    ])

class CocoTransform:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        image = self.transforms(image)
        return image, target


In [ ]:
%%writefile train_faster_rcnn.py
import torch
import torchvision
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import tempfile
import json
import numpy as np

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Needed for batching
def collate_fn(batch):
    return tuple(zip(*batch))

# Standard image transform
def get_transform():
    return T.Compose([
        T.ToTensor()
    ])

# Wrapper so transforms accept (image, target)
class CocoTransform:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        image = self.transforms(image)
        return image, target

# Load Faster-RCNN model
def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

# COCO evaluation wrapper
def evaluate(model, dataloader, ann_file, device):
    model.eval()
    coco_gt = COCO(ann_file)
    results = []

    with torch.no_grad():
        for imgs, targets in dataloader:
            imgs = [img.to(device) for img in imgs]
            outputs = model(imgs)

            for target, output in zip(targets, outputs):
                image_id = target["image_id"].item()

                for box, score, label in zip(output["boxes"], output["scores"], output["labels"]):
                    x1, y1, x2, y2 = box.tolist()
                    results.append({
                        "image_id": image_id,
                        "category_id": int(label),
                        "bbox": [x1, y1, x2 - x1, y2 - y1],
                        "score": float(score)
                    })

    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        json.dump(results, f)
        temp_json = f.name

    coco_dt = coco_gt.loadRes(temp_json)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval.stats[1]  # mAP@50

def train():
    train_img = "/content/coco_dataset/images/train"
    val_img = "/content/coco_dataset/images/val"
    train_ann = "/content/coco_dataset/annotations/instances_train.json"
    val_ann = "/content/coco_dataset/annotations/instances_val.json"

    print("loading annotations...")
    train_ds = CocoDetection(train_img, train_ann, transforms=CocoTransform(get_transform()))
    val_ds = CocoDetection(val_img, val_ann, transforms=CocoTransform(get_transform()))

    train_dl = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=2, collate_fn=collate_fn)
    val_dl = DataLoader(val_ds, batch_size=4, shuffle=False, num_workers=2, collate_fn=collate_fn)

    num_classes = 1 + 8  # background + 8 disease classes
    model = get_model(num_classes).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    best_map50 = 0.0

    for epoch in range(5):
        print(f"Epoch {epoch+1}")
        model.train()

        for imgs, targets in train_dl:
            imgs = [img.to(DEVICE) for img in imgs]
            formatted = []

            for t in targets:
                boxes = torch.tensor(t["boxes"], dtype=torch.float32).to(DEVICE)
                labels = torch.tensor(t["labels"], dtype=torch.int64).to(DEVICE)
                formatted.append({"boxes": boxes, "labels": labels})

            loss_dict = model(imgs, formatted)
            loss = sum(loss_dict.values())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Evaluating...")
        map50 = evaluate(model, val_dl, val_ann, DEVICE)

        if map50 > best_map50:
            best_map50 = map50
            torch.save(model.state_dict(), "/content/faster_rcnn_best.pth")
            print("Saved BEST model!")

    print("Training completed.")
    print("Best mAP50:", best_map50)

if __name__ == "__main__":
    train()


Overwriting train_faster_rcnn.py


In [ ]:
%%writefile train_faster_rcnn.py
import torch
import torchvision
import torchvision.transforms as T
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from pycocotools.cocoeval import COCOeval
from pycocotools.coco import COCO
import tempfile, json, numpy as np

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(batch):
    return tuple(zip(*batch))

def get_transform():
    return T.Compose([T.ToTensor()])

class CocoTransform:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        image = self.transforms(image)
        return image, target

def get_model(num_classes):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

def evaluate(model, dataloader, ann_file, device):
    model.eval()
    coco_gt = COCO(ann_file)
    results = []

    with torch.no_grad():
        for imgs, targets in dataloader:
            imgs = [img.to(device) for img in imgs]
            outputs = model(imgs)

            for t_list, output in zip(targets, outputs):
                image_id = t_list[0]["image_id"]  # same for all anns in that image
                image_id = int(image_id)

                for box, score, label in zip(output["boxes"], output["scores"], output["labels"]):
                    x1, y1, x2, y2 = box.tolist()
                    results.append({
                        "image_id": image_id,
                        "category_id": int(label),
                        "bbox": [x1, y1, x2 - x1, y2 - y1],
                        "score": float(score)
                    })

    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        json.dump(results, f)
        pred_file = f.name

    coco_dt = coco_gt.loadRes(pred_file)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    return coco_eval.stats[1]  # mAP50

def train():
    train_img = "/content/coco_dataset/images/train"
    val_img   = "/content/coco_dataset/images/val"
    train_ann = "/content/coco_dataset/annotations/instances_train.json"
    val_ann   = "/content/coco_dataset/annotations/instances_val.json"

    print("Loading datasets...")

    train_ds = CocoDetection(train_img, train_ann, transforms=CocoTransform(get_transform()))
    val_ds   = CocoDetection(val_img, val_ann, transforms=CocoTransform(get_transform()))

    train_dl = DataLoader(train_ds, batch_size=4, shuffle=True,
                          num_workers=2, collate_fn=collate_fn)
    val_dl   = DataLoader(val_ds, batch_size=4, shuffle=False,
                          num_workers=2, collate_fn=collate_fn)

    num_classes = 1 + 8  # background + 8 disease classes
    model = get_model(num_classes).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    best_map50 = 0.0

    for epoch in range(5):
        print(f"\n========== EPOCH {epoch+1} ==========")
        model.train()

        for imgs, targets in train_dl:
            imgs = [img.to(DEVICE) for img in imgs]
            formatted = []

            for t_list in targets:
                boxes = []
                labels = []

                for ann in t_list:
                    x, y, w, h = ann["bbox"]
                    boxes.append([x, y, x+w, y+h])
                    labels.append(ann["category_id"])

                if len(boxes) == 0:
                    boxes = torch.zeros((0, 4), dtype=torch.float32)
                    labels = torch.zeros((0,), dtype=torch.int64)
                else:
                    boxes = torch.tensor(boxes, dtype=torch.float32)
                    labels = torch.tensor(labels, dtype=torch.int64)

                formatted.append({
                    "boxes": boxes.to(DEVICE),
                    "labels": labels.to(DEVICE)
                })

            loss_dict = model(imgs, formatted)
            loss = sum(loss_dict.values())

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Evaluating...")
        map50 = evaluate(model, val_dl, val_ann, DEVICE)

        if map50 > best_map50:
            best_map50 = map50
            torch.save(model.state_dict(), "/content/faster_rcnn_best.pth")
            print("🔥 Saved NEW BEST model (mAP50={:.3f})".format(map50))

    print("\nTraining Completed.")
    print("Best mAP50:", best_map50)

if __name__ == "__main__":
    train()


Overwriting train_faster_rcnn.py


In [ ]:
!python train_faster_rcnn.py


Loading datasets...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

========== EPOCH 1 ==========
Evaluating...
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.01s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.07s).
Accumulating evaluation results...
DONE (t=0.06s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.012
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.040
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.002
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.010
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 

mAP@0.50:

Epoch 1 → 0.040

Epoch 2 → 0.117

Epoch 3 → 0.129

Epoch 4 → 0.166 ⭐ Best so far!

mAP@[0.50:0.95]:

Epoch 1 → 0.012

Epoch 2 → 0.050

Epoch 3 → 0.054

Epoch 4 → 0.085 🔥

Recall (AR@100):

Epoch 1 → 0.088

Epoch 2 → 0.168

Epoch 3 → 0.280

Epoch 4 → 0.298

📌 Best mAP@0.50 = 0.1921

This is very strong for this dataset.

📌 Best mAP@[0.50:0.95] = 0.093

In [ ]:
from google.colab import files
files.download('/content/yolo_train/exp_final/weights/best.pt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
files.download('/content/faster_rcnn_best.pth')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import zipfile

zip_path = "/content/chestxray_models.zip"

with zipfile.ZipFile(zip_path, 'w') as zipf:
    zipf.write('/content/yolo_train/exp_final/weights/best.pt', arcname='best_yolo.pt')
    zipf.write('/content/faster_rcnn_best.pth', arcname='best_faster_rcnn.pth')

print("ZIP created:", zip_path)


ZIP created: /content/chestxray_models.zip


In [ ]:
from google.colab import files
files.download('/content/chestxray_models.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>